# Food Delivery Dataset — Solution Notebook
This notebook loads three files (`orders.csv`, `users.json`, `restaurants.sql`), merges them using LEFT JOINs, and answers the MCQs + numerical + fill-in-the-blanks.

**Files expected in the same folder / paths:**
- `orders.csv`
- `users.json`
- `restaurants.sql`


In [ ]:
import pandas as pd, json, sqlite3
import numpy as np

orders_path = 'orders.csv'
users_path = 'users.json'
restaurants_path = 'restaurants.sql'

# 1) Load CSV
orders = pd.read_csv(orders_path)
orders.head()

In [ ]:
# 2) Load JSON
with open(users_path, 'r', encoding='utf-8') as f:
    users = json.load(f)
users_df = pd.DataFrame(users)
users_df.head()

In [ ]:
# 3) Load SQL (restaurants.sql) into an in-memory SQLite DB
conn = sqlite3.connect(':memory:')
with open(restaurants_path, 'r', encoding='utf-8') as f:
    conn.executescript(f.read())
restaurants_df = pd.read_sql_query('SELECT * FROM restaurants', conn)
restaurants_df.head()

In [ ]:
# 4) Clean + Merge (LEFT JOINs)
orders_clean = orders.copy()
if 'restaurant_name' in orders_clean.columns:
    orders_clean = orders_clean.drop(columns=['restaurant_name'])

# parse date
orders_clean['order_date'] = pd.to_datetime(orders_clean['order_date'], format='%d-%m-%Y', errors='coerce')

final_df = (
    orders_clean
    .merge(users_df, on='user_id', how='left')
    .merge(restaurants_df, on='restaurant_id', how='left')
)
final_df.shape, final_df.head()

In [ ]:
# Save final merged dataset
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print('Saved: final_food_delivery_dataset.csv')

## MCQs — Computation

In [ ]:
def rating_bucket(r):
    if pd.isna(r): return np.nan
    if 3.0 <= r <= 3.5: return '3.0–3.5'
    if 3.6 <= r <= 4.0: return '3.6–4.0'
    if 4.1 <= r <= 4.5: return '4.1–4.5'
    if 4.6 <= r <= 5.0: return '4.6–5.0'
    return 'other'

final_df['rating_range'] = final_df['rating'].apply(rating_bucket)
final_df['quarter'] = 'Q' + final_df['order_date'].dt.quarter.astype('Int64').astype(str)

gold_df = final_df[final_df['membership'].str.lower()=='gold']

# Q1
q1 = gold_df.groupby('city')['total_amount'].sum().sort_values(ascending=False)
q1

In [ ]:
# Q2
q2 = final_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
q2

In [ ]:
# Q3
user_totals = final_df.groupby('user_id')['total_amount'].sum()
q3_count = int((user_totals > 1000).sum())
q3_count

In [ ]:
# Q4
q4 = final_df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)
q4

In [ ]:
# Q5
q5 = gold_df.groupby('city')['total_amount'].mean().sort_values(ascending=False)
q5

In [ ]:
# Q6 (lowest #distinct restaurants; among ties pick highest revenue)
cuisine_rest_counts = final_df.groupby('cuisine')['restaurant_id'].nunique()
min_count = cuisine_rest_counts.min()
candidates = cuisine_rest_counts[cuisine_rest_counts==min_count].index
cuisine_revs = final_df.groupby('cuisine')['total_amount'].sum()
q6_choice = cuisine_revs.loc[candidates].sort_values(ascending=False)
cuisine_rest_counts, q6_choice

In [ ]:
# Q7
pct_gold = round((gold_df.shape[0]/final_df.shape[0])*100)
pct_gold

In [ ]:
# Q8
rest_stats = final_df.groupby('restaurant_name').agg(
    order_count=('order_id','count'),
    avg_order_value=('total_amount','mean')
)
rest_stats[rest_stats['order_count']<20].sort_values('avg_order_value', ascending=False).head(10)

In [ ]:
# Q9
combo_rev = final_df.groupby(['membership','cuisine'])['total_amount'].sum().sort_values(ascending=False)
combo_rev

In [ ]:
# Q10
q10 = final_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)
q10

## Numerical answers — Computation

In [ ]:
n1_gold_orders = int(gold_df.shape[0])
n2_hyd_rev = int(round(final_df.loc[final_df['city']=='Hyderabad','total_amount'].sum()))
n3_distinct_users = int(final_df['user_id'].nunique())
n4_gold_aov = round(float(gold_df['total_amount'].mean()), 2)
n5_orders_rating_ge_45 = int(final_df[final_df['rating']>=4.5].shape[0])
top_gold_city = gold_df.groupby('city')['total_amount'].sum().sort_values(ascending=False).index[0]
n6_orders_top_gold_city = int(gold_df[gold_df['city']==top_gold_city].shape[0])

n1_gold_orders, n2_hyd_rev, n3_distinct_users, n4_gold_aov, n5_orders_rating_ge_45, top_gold_city, n6_orders_top_gold_city

## Final Answers (ready to paste into the form)

In [ ]:
answers = {
 'MCQ1_city_highest_gold_revenue': q1_city_gold_rev,
 'MCQ2_cuisine_highest_avg_order_value': q2_cuisine_highest_aov,
 'MCQ3_distinct_users_gt_1000_total_orders': q3_distinct_users_gt1000,
 'MCQ4_rating_range_highest_total_revenue': q4_rating_range_highest_rev,
 'MCQ5_city_highest_avg_order_value_gold': q5_city_gold_highest_aov,
 'MCQ6_cuisine_lowest_distinct_restaurants_significant_revenue': q6_cuisine_lowest_rest_significant,
 'MCQ7_percent_orders_by_gold_members': q7_pct_orders_gold,
 'MCQ8_restaurant_highest_aov_under_20_orders': q8_restaurant,
 'MCQ9_highest_revenue_combo_membership_cuisine': f"{q9_combo[0]} + {q9_combo[1]}",
 'MCQ10_quarter_highest_total_revenue': q10_quarter,
 'NUM1_total_orders_by_gold': n1_gold_orders,
 'NUM2_total_revenue_hyderabad_rounded': n2_hyd_rev,
 'NUM3_distinct_users_with_orders': n3_distinct_users,
 'NUM4_gold_avg_order_value_2dp': n4_gold_aov,
 'NUM5_orders_restaurants_rating_ge_4_5': n5_orders_rating_ge_45,
 'NUM6_orders_in_top_gold_revenue_city': n6_orders_top_gold_city,
}
pd.Series(answers)

## Fill-in-the-blanks

In [ ]:
fill_answers = {
 'join_orders_users': 'user_id',
 'cuisine_rating_format': 'SQL',
 'final_rows': int(final_df.shape[0]),
 'no_user_match': 'NaN',
 'pandas_combine': 'merge',
 'membership_origin': 'users.json',
 'join_orders_restaurants': 'restaurant_id',
 'food_type_column': 'cuisine',
 'details_multiple_orders': 'multiple',
}
pd.Series(fill_answers)